# Создание таблиц для анализа данных по финансовым рынкам индекса snp500

In [7]:
import pandas as pd
import yfinance as yf
import yahoo_fin.stock_info as si

In [8]:
pd.options.display.max_rows = 10

In [9]:
path = "YTickers.csv" # тикеры индекса snp500
ticker = pd.read_csv(path)

In [10]:
tickers = ticker.sort_values(by="Tickers")["Tickers"].tolist()

In [11]:
data = yf.download(tickers, start="2012-01-01", end="2021-01-01")

[*********************100%***********************]  505 of 505 completed

4 Failed downloads:
- BRK.B: No data found, symbol may be delisted
- OGN: Data doesn't exist for startDate = 1325350800, endDate = 1609434000
- BF.B: No data found for this date range, symbol may be delisted
- KSU: No data found, symbol may be delisted


## Таблица 6. Исторические данные о котировках акций на момент закрытия (Quotes)

In [12]:
Quotes = data.copy()['Close'].stack()

In [13]:
Quotes = Quotes.reset_index()

In [14]:
Quotes.columns = ['Date', 'Ticker', 'Close']

In [15]:
Quotes = Quotes.set_index('Ticker')

In [16]:
Quotes

,Date,Close
Ticker,,
A,2012-01-03,26.094421
AAL,2012-01-03,5.120000
AAP,2012-01-03,69.099998
AAPL,2012-01-03,14.686786
ABC,2012-01-03,38.070000
...,...,...
YUM,2020-12-31,108.559998
ZBH,2020-12-31,154.089996
ZBRA,2020-12-31,384.329987


## Таблица 3. Справочник показателей о компании (InfoS)

In [17]:
info = pd.read_csv("Yinfo.csv")

In [18]:
info = info.drop(["Unnamed: 0"], axis=1)

In [19]:
info = info.set_index("Ticker")

In [20]:
info.columns=["Indicator", "Value"]
info

,Indicator,Value
Ticker,,
MMM,zip,55144-1000
MMM,sector,Industrials
MMM,fullTimeEmployees,95000
MMM,longBusinessSummary,"3M Company develops, manufactures, and markets..."
MMM,city,St. Paul
...,...,...
ZTS,dividendYield,0.0049
ZTS,bidSize,800
ZTS,dayHigh,206.085


In [21]:
info = info.sort_values(by="Ticker")

In [22]:
infoS = pd.DataFrame(info["Indicator"] \
                             .unique()) \
                             .reset_index() \
                             .rename(columns = {'index': 'id', 0 : 'Name'}) \
                             .set_index("id")
infoS

,Name
id,
0,sector
1,forwardPE
2,lastCapGain
3,shortPercentOfFloat
4,sharesShortPriorMonth
...,...
164,exchangeDataDelayedBy
165,postMarketPrice
166,exchangeName


## Таблица 5. Информация о компании (Info)

In [23]:
Info = pd.merge(info.reset_index(), infoS.reset_index(), left_on="Indicator", right_on="Name", how="left") \
    .set_index("Ticker") \
    .sort_values(by="Ticker")[["id", "Value"]]
Info

,id,Value
Ticker,,
A,0,Healthcare
A,97,9.614183
A,98,16700
A,99,Santa Clara
A,100,800 227 9770
...,...,...
ZTS,50,800
ZTS,129,3.71
ZTS,56,NaN


## Таблица 2. Справочник сектор (Sector)

In [24]:
Sector = pd.DataFrame(info[info.Indicator == "sector"]["Value"] \
                             .unique()) \
                             .reset_index() \
                             .rename(columns = {'index': 'idSector', 0 : 'Name'}) \
                             .set_index("idSector")
Sector

,Name
idSector,
0,Healthcare
1,Industrials
2,Consumer Cyclical
3,Technology
4,Consumer Defensive
...,...
6,Financial Services
7,Basic Materials
8,Real Estate


## Таблица 1. Справочник компаний (Companies)

In [25]:
names = info[info.Indicator == "longName"][["Value"]].rename(columns={"Value": "Name"})

In [26]:
ticker_id_sector = pd.merge(info[info.Indicator == "sector"][["Value"]].reset_index(), Sector.reset_index(), left_on="Value", right_on="Name")[["Ticker", "idSector"]].set_index("Ticker")

In [27]:
Companies = pd.merge(names, ticker_id_sector, left_index=True, right_index=True, how="inner")

In [28]:
Companies

,Name,idSector
Ticker,,
A,"Agilent Technologies, Inc.",0
AAL,American Airlines Group Inc.,1
AAP,"Advance Auto Parts, Inc.",2
AAPL,Apple Inc.,3
ABBV,AbbVie Inc.,0
...,...,...
YUM,"Yum! Brands, Inc.",2
ZBH,"Zimmer Biomet Holdings, Inc.",0
ZBRA,Zebra Technologies Corporation,3


## Таблица 4. Справочник показателей отчетности (StatementS) 

In [37]:
Statements = pd.DataFrame()
for i in tickers[:10]:
    try:
        Stat = pd.DataFrame(si.get_balance_sheet(f'{i}').stack().reset_index())
        Stat['Ticker'] = f'{i}'
        Stat['Stat'] = 'balance_sheet'
        Statements = pd.concat([Statements, Stat])
    except (ValueError, IndexError, KeyError):
        pass
for i in tickers[:10]:
    try:
        Stat = pd.DataFrame(si.get_income_statement(f'{i}').stack().reset_index())
        Stat['Ticker'] = f'{i}'
        Stat['Stat'] =  'income_statement'
        Statements = pd.concat([Statements, Stat])
    except (ValueError, IndexError, KeyError):
        pass

for i in tickers[:10]:
    try:
        Stat = pd.DataFrame(si.get_cash_flow(f'{i}').stack().reset_index())
        Stat['Ticker'] = f'{i}'
        Stat['Stat'] = 'cash_flow'
        Statements = pd.concat([Statements, Stat])
    except (ValueError, IndexError, KeyError):
        pass

In [38]:
StatementS_balance = pd.DataFrame(Statements[Statements['Stat'] == 'balance_sheet']['Breakdown'].unique()).reset_index().rename(columns = {'index': 'id', 0 : 'Name'})
StatementS_balance['Stat'] = 'balance_sheet'
StatementS_income = pd.DataFrame(Statements[Statements['Stat'] == 'income_statement']['Breakdown'].unique()).reset_index().rename(columns = {'index': 'id', 0 : 'Name'})
StatementS_income['Stat'] = 'income_statement'
StatementS_cash = pd.DataFrame(Statements[Statements['Stat'] == 'cash_flow']['Breakdown'].unique()).reset_index().rename(columns = {'index': 'id', 0 : 'Name'})
StatementS_cash['Stat'] = 'cash_flow'

StatementS = pd.concat([StatementS_balance, StatementS_income, StatementS_cash], axis = 0).reset_index()
StatementS = StatementS[['Stat', 'Name']].reset_index().rename(columns = {'index' : 'id'})
StatementS

,id,Stat,Name
0,0,balance_sheet,intangibleAssets
1,1,balance_sheet,capitalSurplus
2,2,balance_sheet,totalLiab
3,3,balance_sheet,totalStockholderEquity
4,4,balance_sheet,otherCurrentLiab
...,...,...,...
62,62,cash_flow,changeToInventory
63,63,cash_flow,changeToAccountReceivables
64,64,cash_flow,otherCashflowsFromFinancingActivities
65,65,cash_flow,changeToNetincome


## Таблица 7. Финансовая отчетность компаний (Statements) 

In [40]:
Statements = Statements.rename(columns = {'Breakdown': 'Name', 'endDate' : 'Date', 0 : 'Value'})
Statements = Statements.merge(StatementS, on=['Name', 'Stat'])
Statements = Statements.rename(columns = {'id': 'Ind'})[['Ticker', 'Date', 'Ind', 'Value']]
Statements

,Ticker,Date,Ind,Value
0,A,2021-10-31,0,981000000.0
1,A,2020-10-31,0,831000000.0
2,A,2019-10-31,0,1107000000.0
3,A,2018-10-31,0,491000000.0
4,AAL,2021-12-31,0,1988000000.0
...,...,...,...,...
2268,ACN,2018-08-31,66,-619187000
2269,ADBE,2021-12-03,66,-348000000.0
2270,ADBE,2020-11-27,66,-419000000.0
2271,ADBE,2019-11-29,66,-395000000.0


In [41]:
Statements = Statements.set_index("Ticker")

## Сохраняем данные

In [36]:
Companies.to_excel('Companies.xlsx')
Sector.to_excel('Sector.xlsx')
infoS.to_excel('InfoS.xlsx')
StatementS.to_excel('StatementS.xlsx')

Info.to_csv('Info')
Quotes.to_csv('Quotes')
Statements.to_csv('Statements')